In [1]:
import json
import os
from tdmq.client import Client
from datetime import datetime, timedelta

INFO:tdmq.db:Logging is active.
INFO:tdmq.client.client:Logging is active.


The Italian DPC (FIXME) provides a service that every 5 minutes releases a new dataset build as a mosaic of data coming from DPC meteo radars.
Each record will consist of a 1400 by 1200 'image' with two float32 bands, VMI and SRI that represent, respectively, the precipitation (in mm/hour) and the maximum signal intensity (in db? FIXME).
The section 'geomapping' contains information needed to map the image grid to geography.

In [2]:
desc = {
    "id": "tdm/tiledb_sensor_foo",
    "alias": "Mosaic of dpc meteo radars",
    "entity_category": "Radar",
    "entity_type": "MeteoRadarMosaic",
    "default_footprint": {
         "coordinates": [[
             [4.537000517753033, 47.856095810774605, 0.0, 1.0],
             [4.537000517753033, 35.07686201381699, 0.0, 1.0],
             [20.436762466677894, 35.07686201381699, 0.0, 1.0],
             [20.436762466677894, 47.856095810774605, 0.0, 1.0],
             [4.537000517753033, 47.856095810774605, 0.0, 1.0]]],
         "type": "Polygon" },
    "stationary": True,
    "controlledProperties": ["VMI", "SRI"], 
    "shape": [1400, 1200],
    "storage": "tiledb",
    "geomapping": {
        'SRID': 'EPSG:4326',
        'grid': {'xsize': 1200, 'ysize': 1400},
        'ModelTransformation': [
            [0.013249801624104052, 0.0, 0.0, 4.537000517753033],
            [0.0, -0.009128024140684008, 0.0, 47.856095810774605],
            [0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1.0]]
    },
    "description": {
        "type": "meteoRadar",
        "brandName": "DPC",
        "modelName": "dpc-radar-mosaic",
        "manufacturerName": "Dipartimento Protezione Civile",
        "category": ["sensor"],
        "function": ["sensing"],
        "reference": "http://www.protezionecivile.gov.it/attivita-rischi/meteo-idro/attivita/previsione-prevenzione/centro-funzionale-centrale-rischio-meteo-idrogeologico/monitoraggio-sorveglianza/mappa-radar" }
}


In [3]:
import tiledb
# FIXME 
c = Client(tiledb_ctx=tiledb.Ctx({'vfs.hdfs.username': 'root'}))

In [4]:
import numpy as np

In [5]:
s = c.register_source(desc)

DEBUG:tdmq.client.client:registering sources id=tdm/tiledb_sensor_foo
DEBUG:tdmq.client.client:sources (tdm/tiledb_sensor_foo) registered as tdmq_id=d53cb9e7-718a-5a1a-a803-1df48dee2e6b
DEBUG:tdmq.client.client:attempting creation of hdfs://namenode:8020/arrays/d53cb9e7-718a-5a1a-a803-1df48dee2e6b
DEBUG:tdmq.client.client:trying domain creation for hdfs://namenode:8020/arrays/d53cb9e7-718a-5a1a-a803-1df48dee2e6b
DEBUG:tdmq.client.client:trying attribute creation for hdfs://namenode:8020/arrays/d53cb9e7-718a-5a1a-a803-1df48dee2e6b
DEBUG:tdmq.client.client:trying ArraySchema creation for hdfs://namenode:8020/arrays/d53cb9e7-718a-5a1a-a803-1df48dee2e6b
DEBUG:tdmq.client.client:trying creation on disk of hdfs://namenode:8020/arrays/d53cb9e7-718a-5a1a-a803-1df48dee2e6b
DEBUG:tdmq.client.client:hdfs://namenode:8020/arrays/d53cb9e7-718a-5a1a-a803-1df48dee2e6b successfully created.
DEBUG:tdmq.client.client:new managed object d53cb9e7-718a-5a1a-a803-1df48dee2e6b


In [6]:
def create_data_frame(shape, properties, slot):
    data = {}
    for p in properties:
        data[p] = np.full(shape, slot, dtype=np.float32)
    return data

def ingest_records(s, N):
    now = datetime.now()
    t = now
    dt = timedelta(seconds=300)
    for slot in range(N):
        data = create_data_frame(s.shape, s.controlled_properties, slot)
        s.ingest(t, data, slot)
        t += dt
    return now, dt

Fake data acquisition

In [7]:
N = 10

In [8]:
timebase, dt = ingest_records(s, N)

In [9]:
ts = s.timeseries()

DEBUG:tdmq.client.client:get_timeseries(d53cb9e7-718a-5a1a-a803-1df48dee2e6b, {})


In [10]:
ts_times, D = ts[:]

In [11]:
ts_times

array([datetime.datetime(2019, 8, 6, 20, 17, 20, 896300),
       datetime.datetime(2019, 8, 6, 20, 22, 20, 896300),
       datetime.datetime(2019, 8, 6, 20, 27, 20, 896300),
       datetime.datetime(2019, 8, 6, 20, 32, 20, 896300),
       datetime.datetime(2019, 8, 6, 20, 37, 20, 896300),
       datetime.datetime(2019, 8, 6, 20, 42, 20, 896300),
       datetime.datetime(2019, 8, 6, 20, 47, 20, 896300),
       datetime.datetime(2019, 8, 6, 20, 52, 20, 896300),
       datetime.datetime(2019, 8, 6, 20, 57, 20, 896300),
       datetime.datetime(2019, 8, 6, 21, 2, 20, 896300)], dtype=object)

In [12]:
D.keys()

odict_keys(['VMI', 'SRI'])

In [13]:
D['VMI'].shape, D['SRI'].shape

((10, 1400, 1200), (10, 1400, 1200))

In [14]:
ts_times, D = ts[3:6]

In [15]:
ts_times

array([datetime.datetime(2019, 8, 6, 20, 32, 20, 896300),
       datetime.datetime(2019, 8, 6, 20, 37, 20, 896300),
       datetime.datetime(2019, 8, 6, 20, 42, 20, 896300)], dtype=object)

In [16]:
D['VMI'].shape, D['SRI'].shape

((3, 1400, 1200), (3, 1400, 1200))

In [17]:
ts_times, D = ts[4,100:130, 40:60]

In [18]:
ts_times

datetime.datetime(2019, 8, 6, 20, 37, 20, 896300)

In [19]:
D['VMI'].shape, D['SRI'].shape

((30, 20), (30, 20))

In [20]:
c.deregister_source(s)

DEBUG:tdmq.client.client:deregistering d53cb9e7-718a-5a1a-a803-1df48dee2e6b <tdmq.client.sources.NonScalarSource object at 0x7f95fc70d710>
DEBUG:tdmq.client.client:removing from managed_objects
